In [1]:
import pandas as pd
import pymongo
from IPython.display import clear_output
from geopy.geocoders import Nominatim
import json
import re
import string
import concurrent.futures
import requests

In [2]:
# Connect to database
connection_string='mongodb://localhost:27017'
client = pymongo.MongoClient(connection_string)
# Define database for storage
db = client.dice_db

In [3]:
# # Drop Duplicates from database
# import json
# def drop_db_duplicates(db_table):
#     # Retrieve data for analysis
#     actual_collection = db_table.find()
#     #get mongoDB data into a dataframe
#     actual_collection_df = pd.DataFrame(list(actual_collection))
#     #exclude unique index and drop duplicates
#     actual_collection_df = actual_collection_df.loc[:, actual_collection_df.columns != '_id']
#     actual_collection_df = actual_collection_df.drop_duplicates()
#     #prepare records to load into database
#     records = json.loads(actual_collection_df.T.to_json()).values()
#     #insert into database unique values
#     db_table.drop()
#     db_table
#     db_table.insert(records)

In [4]:
# drop_db_duplicates(db.jobs_test)

In [5]:
jobs_collection = db.jobs.find()
jobs_df = pd.DataFrame(list(jobs_collection))
jobs_df

,_id,job_title,job_company,job_salary,job_location,job_date,job_description,location_latitude,location_longitude,job_type
0,5fd6a28bca0e57fbbf2284e3,Business Analytics Director,University of Kentucky,Depends on Experience,"Lexington, KY, USA",2020-12-12,\nThe University of Kentucky’s Institutional R...,NaN,NaN,NaN
1,5fd6a28bca0e57fbbf2284e4,Associate Software Engineer III,Penske,Depends on Experience,"Reading, PA, USA",2020-12-12,\nSummary Statement:Working with a diverse tea...,NaN,NaN,NaN
2,5fd6a28bca0e57fbbf2284e5,Developer,Healh Care Logistics,"$70,000 - $85,000","Grove City, OH, USA",2020-12-11,\nTitle: Developer ...,NaN,NaN,NaN
3,5fd6a28bca0e57fbbf2284e6,ServiceNow Solution Architect,Saxon Global Inc.,$open,"Charlotte, NC, USA",2020-12-13,"\nHi, Please go through the job description an...",NaN,NaN,NaN
4,5fd6a28bca0e57fbbf2284e7,Full Time HRIS Developer,Saxon Global Inc.,Depends On Experience,"Nashville, TN, USA",2020-12-13,"\nLocation: Nashville TN, Memphis TN , Ridgela...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
27421,5fd81d49eb016bc81b4f95fe,Junior Mobile Developer - C#,"eXcell, a division of CompuCom Systems, Inc.",BASED ON EXPERIENCE,"Kent, WA, USA",2020-12-14,"\nOur client is seeking a full-time, passionat...",47.3827,-122.227,Full Time
27422,5fd81d4beb016bc81b4f95ff,Senior Salesforce Business Analyst,Synapse Business Systems,BASED ON EXPERIENCE,"Arlington, VA, USA",2020-12-14,\nSenior Salesforce Business Analyst - Need Se...,32.7019,-97.1056,"Contract W2, Contract Independent, C2H W2, C2H..."
27423,5fd81d4deb016bc81b4f9600,Salesforce Business Analyst,Synapse Business Systems,BASED ON EXPERIENCE,"Washington, DC, USA",2020-12-14,\nSENIOR BUSINESS ANALYST with Active Secret C...,38.895,-77.0366,"Contract W2, Contract Independent, C2H W2, C2H..."
27424,5fd81d50eb016bc81b4f9601,Need Data Lead for Washington DC,Synapse Business Systems,BASED ON EXPERIENCE,"Washington, DC, USA",2020-12-14,\n Job Position : Data LeadLocation : Washingt...,38.895,-77.0366,"Contract W2, Contract Independent, C2H W2, C2H..."


In [6]:
jobs_df.loc[:, jobs_df.columns != '_id'].drop_duplicates().describe()

,job_title,job_company,job_salary,job_location,job_date,job_description,location_latitude,location_longitude,job_type
count,27426,27426,27426,27426,27426,27426,7025.000000,7025.000000,7025
unique,12334,530,1098,2326,47,17347,848.000000,848.000000,21
top,REMOTE Full Stack LAMP Developer - UI/UX Design,Apex Systems,,"New York, NY, USA",2020-12-13,\nCX Researcher / Journey Mapper for Global Se...,38.894154,-77.431151,Full Time
freq,291,2567,19545,869,15375,29,322.000000,322.000000,6775


In [7]:
jobs_df["job_date"].unique()

array(['2020-12-12', '2020-12-11', '2020-12-13', '2020-12-10',
       '36 minutes ago', '37 minutes ago', '39 minutes ago',
       '41 minutes ago', '42 minutes ago', '23 minutes ago',
       '24 minutes ago', '30 minutes ago', '31 minutes ago',
       '32 minutes ago', '35 minutes ago', '38 minutes ago',
       '40 minutes ago', '45 minutes ago', '49 minutes ago',
       '50 minutes ago', '51 minutes ago', '58 minutes ago',
       '47 minutes ago', '48 minutes ago', '1 hour ago', '2 hours ago',
       '33 minutes ago', '2020-12-09', '2020-12-08', '2020-12-05',
       '2020-12-04', '2020-12-03', '2020-12-01', '2020-11-21',
       '2020-11-20', '2020-11-19', '2020-11-18', '2020-11-17',
       '2020-11-14', '2020-11-05', '2020-11-04', '2020-10-30',
       '2020-10-24', '2020-10-23', '2020-10-16', 'None', '2020-12-14'],
      dtype=object)

In [8]:
# Apply a first round of text cleaning techniques
re.DOTALL
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', ' ', text)

    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', ' ', text)
    text = re.sub('\n', ' ', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [9]:
# Apply the data cleaning
jobs_clean_df = jobs_df
jobs_clean_df = pd.DataFrame(jobs_clean_df["job_description"].apply(round1))

In [10]:
# See description before transformation
jobs_df.loc[1, "job_description"]

'\nSummary Statement:Working with a diverse team, you will participate in all aspects of the software development lifecycle. You will get an opportunity to design functions and modules and be responsible for delivering the next generation of innovative applications to our customers\xa0What You Will Be Doing:As an Associate Software Engineer III, you will be part of a team that develops our next generation of applications while enhancing and supporting our existing applications. You will be involved in reviewing, interpreting & developing systems in accordance with user requirements & troubleshooting issues as they occur. Working with the business and other software engineers/leads, you will develop an understanding of the requirements and translate them into technical design to develop and implement the solution. You will also collaborate with engineers and architects to ensure optimization, offering your input into the requirements and potential alternative courses of action.\xa0Pensk

In [11]:
# See description after transformation
jobs_clean_df.loc[1, "job_description"]

' summary statement working with a diverse team  you will participate in all aspects of the software development lifecycle  you will get an opportunity to design functions and modules and be responsible for delivering the next generation of innovative applications to our customers\xa0what you will be doing as an associate software engineer iii  you will be part of a team that develops our next generation of applications while enhancing and supporting our existing applications  you will be involved in reviewing  interpreting   developing systems in accordance with user requirements   troubleshooting issues as they occur  working with the business and other software engineers leads  you will develop an understanding of the requirements and translate them into technical design to develop and implement the solution  you will also collaborate with engineers and architects to ensure optimization  offering your input into the requirements and potential alternative courses of action \xa0penske

In [12]:
key_words = ["data+analyst", "data+scientist", "excel", "python", "pandas", "matplotlib", "sql", "postgresql", "bootstrap", "nosql", \
    "mongodb", "mongo", "javascript", "tableau", "machine+learning", "ml", "scikit+learn", "scikit", "keras", "tensorflow", "pyspark", "natural+language+processing", \
        "nlp", "big+data", "etl", "extract+transform+load", "amazon+web+services", "aws", "rds"]

In [13]:
#clean-up characters in keywords
key_words = map(round1, key_words)
key_words = [key for key in key_words]
key_words

['data analyst',
 'data scientist',
 'excel',
 'python',
 'pandas',
 'matplotlib',
 'sql',
 'postgresql',
 'bootstrap',
 'nosql',
 'mongodb',
 'mongo',
 'javascript',
 'tableau',
 'machine learning',
 'ml',
 'scikit learn',
 'scikit',
 'keras',
 'tensorflow',
 'pyspark',
 'natural language processing',
 'nlp',
 'big data',
 'etl',
 'extract transform load',
 'amazon web services',
 'aws',
 'rds']

In [14]:
jobs_clean_df.loc[55,"job_description"]

' role  data analyst location  winston   salem nc long term contract job description           years of experience in it and banking field with    experience in sql server oracle or other database query language   understand and update business reporting and processes which will reduce duplication of data  reduce data entry into multiple systems and facilitate quicker data driven business decisions and document the details for technology team to understand   work with end users  business analysts and developers to streamline data flowing in to the applications    export and manipulate data from business systems to be used in reports and analysis   setup graphs and presentations to visually represent information using excel and or tableau   experience with creating  documenting  and maintaining processes   develop presentations and produce clear data visualizations for multiple levels of staff  management  and variety of stakeholders   implement methods to improve data reliability and q

In [34]:
# count keywords in description
from collections import Counter
def find_words(key_words, string):
    string = string.split()
    separator = " "
    words_found = {}
    for word in key_words:
        n_gram = len(word.split())
        res = Counter(separator.join(string[idx : idx + n_gram]) for idx in range(len(string) - n_gram + 1))
        print(word, res[word])
        if res[word]>0:
            words_found[word] = res[word]
    # add KeyWordsMatch[%]
    words_found["KeyWordsMatch[%]"] = len(words_found)/len(key_words)
    return words_found

In [35]:
import time
time_start = time.time()
print(find_words(key_words, jobs_clean_df.loc[55,"job_description"]))
print(f"time elapsed {time.time()-time_start}")

data analyst 1
data scientist 0
excel 1
python 0
pandas 0
matplotlib 0
sql 1
postgresql 0
bootstrap 0
nosql 0
mongodb 0
mongo 0
javascript 0
tableau 1
machine learning 0
ml 0
scikit learn 0
scikit 0
keras 0
tensorflow 0
pyspark 0
natural language processing 0
nlp 0
big data 0
etl 0
extract transform load 0
amazon web services 0
aws 0
rds 0
{'data analyst': 1, 'excel': 1, 'sql': 1, 'tableau': 1, 'KeyWordsMatch[%]': 0.13793103448275862}
time elapsed 0.003956317901611328


In [43]:
x = {"x":1,"y":2, **find_words(key_words, jobs_clean_df.loc[55,"job_description"])}
#{**x,**find_words(key_words, jobs_clean_df.loc[55,"job_description"])}
x

data analyst 1
data scientist 0
excel 1
python 0
pandas 0
matplotlib 0
sql 1
postgresql 0
bootstrap 0
nosql 0
mongodb 0
mongo 0
javascript 0
tableau 1
machine learning 0
ml 0
scikit learn 0
scikit 0
keras 0
tensorflow 0
pyspark 0
natural language processing 0
nlp 0
big data 0
etl 0
extract transform load 0
amazon web services 0
aws 0
rds 0


{'x': 1,
 'y': 2,
 'data analyst': 1,
 'excel': 1,
 'sql': 1,
 'tableau': 1,
 'KeyWordsMatch[%]': 0.13793103448275862}

In [36]:
db.jobs_test
db.jobs_test.insert_one(find_words(key_words, jobs_clean_df.loc[55,"job_description"]))

data analyst 1
data scientist 0
excel 1
python 0
pandas 0
matplotlib 0
sql 1
postgresql 0
bootstrap 0
nosql 0
mongodb 0
mongo 0
javascript 0
tableau 1
machine learning 0
ml 0
scikit learn 0
scikit 0
keras 0
tensorflow 0
pyspark 0
natural language processing 0
nlp 0
big data 0
etl 0
extract transform load 0
amazon web services 0
aws 0
rds 0


In [ ]:
# jobs_df = pd.read_csv("ProcessedData.csv")
# jobs_df = jobs_df.drop("Unnamed: 0", axis=1)

In [ ]:
# Function to split the salary text
def split_salary(string):
    
    try:
        amount = re.findall('(\$\d+,\d+|\$\d+)', string)
        salary_min = amount[0]
        salary_min = re.findall("\d+,\d+|\d+",salary_min)[0]
        try:
            salary_max = amount[1]#.group(0)
            salary_max = re.findall("\d+,\d+|\d+",salary_max)[0]
        except:
            salary_max = ""
        um = re.findall('hour|year', string)  
        um = "$/"+um[0]
        return (salary_min, salary_max, um)
    except:
        return ("","","")

In [ ]:
# Split salary text into minimum salary, maximum salary and unit of measurement
salary_list = jobs_df["job_salary"].astype(str).values.tolist()
salary_list = map(split_salary, salary_list)
salary_list = [x for x in salary_list]

In [ ]:
# Record splitted salaries into clean dataframe
salary_list_transposed = zip(*salary_list)
salary_list_transposed = [i for i in salary_list_transposed]
jobs_df["salary_min"] = salary_list_transposed[0]
jobs_df["salary_max"] = salary_list_transposed[1]
jobs_df["UM"] = salary_list_transposed[2]
jobs_df

In [ ]:
# # Insert clean data into new MongoDB table
# #db.jobs_clean.drop()
# clean_records = json.loads(jobs_df.T.to_json()).values()
# db.jobs_clean.insert_many(clean_records)